In [ ]:
import tensorflow as tf
import os
import time
import datetime

from matplotlib import pyplot as plt

In [ ]:
!nvidia-smi -L # what gpu is used right now / (For Google Colab)

In [ ]:
# mount google drive to notebook / (For Google Colab)
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

In [ ]:
# Parameters to change !!!

EPOCHS = 401
BUFFER_SIZE = 2500
BATCH_SIZE = 25
LAMBDA = 100
LEARNING_RATE = 0.0002

# Picture paths. Division into several folders due to timeout returned by Google Colab with long disk access in one path.

imagesTrainPath1 = '/content/gdrive/MyDrive/PythonCode/images/train512_11'
imagesTrainPath2 = '/content/gdrive/MyDrive/PythonCode/images/train512_22'
imagesTrainPath3 = '/content/gdrive/MyDrive/PythonCode/images/train512_33'
imagesTrainPath4 = '/content/gdrive/MyDrive/PythonCode/images/train512_44'
imagesTrainPath5 = '/content/gdrive/MyDrive/PythonCode/images/train512_55'
imagesValPath = '/content/gdrive/MyDrive/PythonCode/images/val512'
log_dir="/content/gdrive/MyDrive/PythonCode/train_logs/"

checkpoint_dir = '/content/gdrive/MyDrive/PythonCode/training_checkpoints'

# Change generator and discriminator layers below if images are in different format than 512x512

In [ ]:
def normalizeImage(inputImage, outputImage):
    inputImage = (inputImage / 127.5) - 1
    outputImage = (outputImage / 127.5) - 1

    return inputImage, outputImage

In [ ]:
def loadImage(imageFile):
    image = tf.io.read_file(imageFile)
    image = tf.image.decode_png(image)

    w = tf.shape(image)[1]

    w = w // 2
    outputImage = image[:, :w, :]
    inputImage = image[:, w:, :]

    inputImage = tf.cast(inputImage, tf.float32)
    outputImage = tf.cast(outputImage, tf.float32)

    return outputImage, inputImage

In [ ]:
def loadImageTrain(imageFile):
    inputImage, outputImage = loadImage(imageFile)
    inputImage, outputImage = normalizeImage(inputImage, outputImage)

    return inputImage, outputImage

In [ ]:
def loadImageTest(imageFile):
    inputImage, outputImage = loadImage(imageFile)
    inputImage, outputImage = normalizeImage(inputImage, outputImage)

    return inputImage, outputImage

In [ ]:
def upsampleLayer(filters, size, shape, applyDropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2, batch_input_shape=shape,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if applyDropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result


def downsampleLayer(filters, size, shape, applyBatchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', batch_input_shape=shape, 
                             kernel_initializer=initializer, use_bias=False))

    if applyBatchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result


def createGenerator():

    inputs = tf.keras.layers.Input(shape=[512,512,3])

    down_stack = [
      downsampleLayer(32, 4, (None, 512, 512, 3), applyBatchnorm = False),
      downsampleLayer(64, 4, (None, 256, 256, 32)),
      downsampleLayer(128, 4, (None, 128, 128, 64)),
      downsampleLayer(256, 4, (None, 64, 64, 128)),
      downsampleLayer(512, 4, (None, 32, 32, 256)),
      downsampleLayer(512, 4, (None, 16, 16, 512)),
      downsampleLayer(512, 4, (None, 8, 8, 512)),
      downsampleLayer(512, 4, (None, 4, 4, 512)),
      downsampleLayer(512, 4, (None, 2, 2, 512)),
    ]

    up_stack = [
      upsampleLayer(512, 4, (None, 1, 1, 512), applyDropout=True),
      upsampleLayer(512, 4, (None, 2, 2, 1024), applyDropout=True),
      upsampleLayer(512, 4, (None, 4, 4, 1024), applyDropout=True),
      upsampleLayer(512, 4, (None, 8, 8, 1024)),
      upsampleLayer(256, 4, (None, 16, 16, 1024)),
      upsampleLayer(128, 4, (None, 32, 32, 512)),
      upsampleLayer(64, 4, (None, 64, 64, 256)),
      upsampleLayer(128, 4, (None, 128, 128, 128)),
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(3, 4,
                                          strides=2,
                                          padding='same',
                                          kernel_initializer=initializer,
                                          activation='tanh')

    x = inputs

    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
def downsLayers(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', 
                             kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result


def createDiscriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tf.keras.layers.Input(shape=[512, 512, 3], name='input_image')
    tar = tf.keras.layers.Input(shape=[512, 512, 3], name='target_image')

    x = tf.keras.layers.concatenate([inp, tar])

    down1 = downsLayers(64, 4, False)(x)
    down2 = downsLayers(128, 4)(down1)
    down3 = downsLayers(256, 4)(down2)
    down4 = downsLayers(512, 4)(down3)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down4)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)

    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)

    return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss


def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

In [ ]:
discriminator = createDiscriminator()
generator = createGenerator()

generator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.5)

In [ ]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
def generate_images(model, test_input, tar, epoch):
    prediction = model(test_input, training=True)
    
    n_samples = 5
    
    if n_samples > BATCH_SIZE:
        n_samples = BATCH_SIZE
    
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.imshow(test_input[i] * 0.5 + 0.5)

    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.imshow(prediction[i] * 0.5 + 0.5)

    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.imshow(tar[i] * 0.5 + 0.5)
        
    filename1 = '/content/gdrive/MyDrive/PythonCode/results/comparision_%06d.png' % (epoch+1)
    plt.savefig(filename1, dpi=450)
    plt.close()

In [ ]:
summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, epoch):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_image, training=True)

        disc_real_output = discriminator([input_image, target], training=True)
        disc_generated_output = discriminator([input_image, gen_output], training=True)

        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))
    with summary_writer.as_default():
        tf.summary.scalar('genTotalLoss', gen_total_loss, step=epoch)
        tf.summary.scalar('genGanLoss', gen_gan_loss, step=epoch)
        tf.summary.scalar('genL1Loss', gen_l1_loss, step=epoch)
        tf.summary.scalar('discLoss', disc_loss, step=epoch)
        
    return gen_total_loss, disc_loss

In [ ]:
def train(train_ds, test_ds):
    
    bat_per_epo = int(number_files / BATCH_SIZE)
    
    for epoch in range(EPOCHS):
        start = time.time()

        for example_input, example_target in test_ds.take(1):
            generate_images(generator, example_input, example_target, epoch)
        print("Epoch: ", epoch)

        for n, (input_image, target) in train_ds.enumerate():
            print(str(int(n)) + "/" + str(bat_per_epo))
            if (n+1) % 100 == 0:
                print('')
            gen_total_loss, disc_loss = train_step(input_image, target, epoch)
            
            g_total_loss = float(gen_total_loss)
            d_loss = float(disc_loss)
            print('Generator loss: {}; Discriminator loss: {}'.format(g_total_loss, d_loss))
        print()

        if (epoch) == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
            generator.save('/content/gdrive/MyDrive/PythonCode/results/generator_' + str(epoch) + '.h5')

            

        print ('Time for epoch {}: {} sec\n'.format(epoch,
                                                        time.time()-start))
    checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
test_dataset = tf.data.Dataset.list_files(imagesValPath + '/*.png')
test_dataset = test_dataset.map(loadImageTest)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
train_dataset1 = tf.data.Dataset.list_files(imagesTrainPath1 + '/*.png')
train_dataset2 = tf.data.Dataset.list_files(imagesTrainPath2 + '/*.png')
train_dataset3 = tf.data.Dataset.list_files(imagesTrainPath3 + '/*.png')
train_dataset4 = tf.data.Dataset.list_files(imagesTrainPath4 + '/*.png')
train_dataset5 = tf.data.Dataset.list_files(imagesTrainPath5 + '/*.png')

train_dataset = train_dataset1.concatenate(train_dataset2).concatenate(train_dataset3).concatenate(train_dataset4).concatenate(train_dataset5)

train_dataset = train_dataset.map(loadImageTrain, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

number_files = len(list(os.listdir(imagesTrainPath1))) + len(list(os.listdir(imagesTrainPath2))) + len(list(os.listdir(imagesTrainPath3))) + len(list(os.listdir(imagesTrainPath4))) + len(list(os.listdir(imagesTrainPath5)))

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

train(train_dataset, test_dataset)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

for example_input, example_target in test_dataset.take(5):
    generate_images(generator, example_input, example_target, 99999)

generator.save('/content/gdrive/MyDrive/PythonCode/results888/generator_finish.h5')